In [1]:
import numpy as np
import cv2
import argparse


In [ ]:
def quaternion_to_matrix(q):
    """Convert a (w, x, y, z) quaternion to a 33603 rotation matrix."""
    w, x, y, z = q
    n = np.linalg.norm([w, x, y, z])
    if n == 0:
        return np.eye(3)
    w, x, y, z = w / n, x / n, y / n, z / n
    R = np.array(
        [
            [1 - 2 * (y * y + z * z), 2 * (x * y - z * w), 2 * (x * z + y * w)],
            [2 * (x * y + z * w), 1 - 2 * (x * x + z * z), 2 * (y * z - x * w)],
            [2 * (x * z - y * w), 2 * (y * z + x * w), 1 - 2 * (x * x + y * y)],
        ],
        dtype=np.float64,
    )
    return R

In [39]:
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.transform import Rotation as R

# ===== Step 1: Load orientation data (quaternions) =====


def load_quaternions(path):
    # Assume Nx4 array of [x, y, z, w]
    return np.loadtxt(path)


def normalize_quaternions(qs):
    return qs / np.linalg.norm(qs, axis=1, keepdims=True)


def smooth_quaternions_gaussian(qs, sigma=2.0):
    smoothed = gaussian_filter1d(qs, sigma=sigma, axis=0, mode="nearest")
    return normalize_quaternions(smoothed)


# ===== Step 2: Apply to video =====


def rotate_equirectangular_frame(frame, rot_matrix):
    h, w = frame.shape[:2]

    # Step 1: Generate direction vectors (lat/lon → 3D unit vectors)
    lon = np.linspace(-np.pi, np.pi, w, endpoint=False)  # [-π, π)
    lat = np.linspace(-np.pi / 2, np.pi / 2, h)  # [-π/2, π/2]
    lon, lat = np.meshgrid(lon, lat)

    x = np.cos(lat) * np.sin(lon)
    y = np.sin(lat)
    z = np.cos(lat) * np.cos(lon)
    dirs = np.stack((x, y, z), axis=-1).reshape(-1, 3)  # (H*W, 3)

    # Step 2: Rotate directions
    dirs_rot = dirs @ rot_matrix.T  # shape: (H*W, 3)

    # Step 3: Convert rotated vectors back to lat/lon
    x_r, y_r, z_r = dirs_rot[:, 0], dirs_rot[:, 1], dirs_rot[:, 2]
    lat_r = np.arcsin(np.clip(y_r, -1, 1))
    lon_r = np.arctan2(x_r, z_r)

    # Step 4: Convert to pixel coordinates
    map_x = ((lon_r + np.pi) / (2 * np.pi)) * w
    map_y = ((lat_r + np.pi / 2) / np.pi) * h
    map_x = map_x.reshape(h, w).astype(np.float32)
    map_y = map_y.reshape(h, w).astype(np.float32)

    # Step 5: Remap the image
    return cv2.remap(
        frame, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP
    )


def process_video(
    input_path,
    quats_smoothed,
    output_path = "stabilized.mp4"
):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = None

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret or frame_idx >= len(quats_smoothed):
            break

        rot = R.from_quat(quats_smoothed[frame_idx])
        rot_matrix = rot.as_matrix()

        warped = rotate_equirectangular_frame(frame, rot_matrix)

        if out is None:
            h, w = warped.shape[:2]
            out = cv2.VideoWriter(
                output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h)
            )

        out.write(warped)
        frame_idx += 1

    cap.release()
    out.release()

In [ ]:
# def stabilize_360_equirect(input_path, rotations, output_path="stabilized_output.mp4"):
#     cap = cv2.VideoCapture(input_path)
#     if not cap.isOpened():
#         raise IOError(f"Cannot open video {input_path}")
#     # Video parameters
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

#     # Prepare output video writer (MP4 with same size and fps)
#     fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#     out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#     # Precompute spherical direction vectors for each pixel (constant)
#     xs = np.linspace(0, width - 1, width, dtype=np.float32)
#     ys = np.linspace(0, height - 1, height, dtype=np.float32)
#     lon = (xs / width) * 2 * np.pi - np.pi  # shape (width,)
#     lat = np.pi / 2 - (ys / height) * np.pi  # shape (height,)
#     lon_map, lat_map = np.meshgrid(lon, lat)  # shape (height, width)
#     # Convert (lat, lon) to 3D unit sphere (x,y,z)
#     X = np.cos(lat_map) * np.sin(lon_map)
#     Y = np.sin(lat_map)
#     Z = np.cos(lat_map) * np.cos(lon_map)
#     dirs = np.stack((X, Y, Z), axis=2)  # shape (height, width, 3)

#     frame_idx = 0
#     for R in rotations:
#         ret, frame = cap.read()
#         if not ret:
#             break  # end of video or error
#         # Convert rotation to 3x3 matrix if needed
#         if isinstance(R, np.ndarray) and R.shape == (3, 3):
#             Rmat = R.astype(np.float64)
#         else:
#             # Assume quaternion (w, x, y, z)
#             q = np.array(R, dtype=np.float64)
#             Rmat = quaternion_to_matrix(q)
#         # Inverse rotation (world->camera)
#         R_inv = Rmat.T  # transpose of orthonormal rotation matrix

#         # Apply R_inv to all direction vectors (vectorized)
#         # Flatten directions to (N,3), apply R_inv, then reshape
#         H, W = height, width
#         dirs_flat = dirs.reshape(-1, 3)  # (H*W, 3)
#         dirs_rot = dirs_flat.dot(R_inv)  # (H*W, 3) rotated directions
#         Xr = dirs_rot[:, 0].reshape(H, W)
#         Yr = dirs_rot[:, 1].reshape(H, W)
#         Zr = dirs_rot[:, 2].reshape(H, W)

#         # Convert rotated vectors back to spherical angles
#         # Clamp y for asin to [-1,1] to avoid numerical issues
#         Yr_clamped = np.clip(Yr, -1.0, 1.0)
#         lat_p = np.arcsin(Yr_clamped)  # ��' in [-��/2, ��/2]
#         lon_p = np.arctan2(Xr, Zr)  # ��' in [-��, ��]

#         # Map spherical angles to equirectangular pixel coordinates
#         map_x = (lon_p / (2 * np.pi) + 0.5) * W  
#         map_y = (0.5 - lat_p / np.pi) * H 

#         # Ensure type float32 for cv2.remap
#         map_x = map_x.astype(np.float32)
#         map_y = map_y.astype(np.float32)

#         # Warp the input frame using remap
#         # BORDER_WRAP ensures horizontal wrap-around for 360�� images.
#         stabilized = cv2.remap(
#             frame,
#             map_x,
#             map_y,
#             interpolation=cv2.INTER_LINEAR,
#             borderMode=cv2.BORDER_WRAP,
#         )

#         out.write(stabilized)
#         frame_idx += 1

#     cap.release()
#     out.release()
#     print(f"Stabilized video saved to {output_path}")

In [40]:
video_file = input("Please input video file")

In [41]:
trajectory_file = input("Please input trajectory file")

In [42]:
def handle_rotation(trajectory_file):
    quaternions = []
    with open(trajectory_file, 'r') as f:
        for line in f:
            line = line.strip()
            elements = line.split(' ')
            q = list(map(float, elements[-5:-1]))  # take 4 last elements minus the last one and convert to float
            quaternions.append(q)
    return quaternions


In [43]:
q = handle_rotation(trajectory_file)
q

[[-3.454675013320278e-05, 0.0007979603223760812, 0.9999996805849731, 1.0],
 [0.00024163116143375026,
  0.000692416267825853,
  0.9999997207830104,
  0.9987938658375327],
 [0.0006568688884688701,
  0.0005883327416245356,
  0.9999995846986452,
  0.9974571043818171],
 [0.001376324479551705,
  0.00048570961507833827,
  0.9999988929001107,
  0.9959919920843915],
 [0.0016290771520523837,
  0.0003845467875219661,
  0.9999985471021243,
  0.9944007925679126],
 [0.0012547924740430138,
  0.0002848444284794196,
  0.9999991444456183,
  0.9926857566261561],
 [0.0008802611906785089,
  0.00018660243748538254,
  0.9999995795723965,
  0.9908491222240174],
 [0.0010745699006642852,
  8.982093632157476e-05,
  0.9999994004107366,
  0.9888931144975103],
 [0.001399508865432712,
  -5.500148028675838e-06,
  0.9999989562092826,
  0.9868199457537674],
 [0.001309285430263005,
  -9.936079727766114e-05,
  0.9999989931269226,
  0.9846318154710408],
 [0.0011565679162961057,
  -0.0001917610316704487,
  0.99999909331219

In [44]:
q = normalize_quaternions(q)
smoothed_quats = smooth_quaternions_gaussian(q, sigma=2)
process_video(video_file, smoothed_quats)